### Train

In [38]:
# import libraries
import os
import torch

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import time
import copy

In [39]:
# move model to GPU if available
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

# batch size, epoch
BATCH_SIZE = 256
EPOCH = 30

cuda


In [40]:
# image to tensor (64 * 64)
transform_base = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

train_dataset = ImageFolder(root='./splitted/train/', transform=transform_base)
val_dataset = ImageFolder(root='./splitted/val/', transform=transform_base)

In [41]:
# DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [42]:
# model
class Net(nn.Module) :

    def __init__(self) :
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=4096, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=33)

    def forward(self, x) :
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=.25)

        # x = x.view(-1, 4096)
        x = torch.flatten(x, 1)

        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

In [43]:
# create model
model_base = Net().to(DEVICE)

# gradient descent
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [44]:
# training function
def train(model, train_loader, optimizer) :
    model.train()
    
    for i, (data, target) in enumerate(train_loader) :
        print(f'Train - [{i+1}/{len(data)}]')
        data = data.to(DEVICE)
        target = target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [45]:
# evaluation function
def evaluate(model, test_loader) :
    model.eval()
    test_loss = 0
    correct = 0

    # not working gradient descent
    with torch.no_grad() :
        for i, (data, target) in enumerate(test_loader) :
            print(f'Evaluation - [{i+1}/{len(data)}]')
            data = data.to(DEVICE)
            target = target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    # average of loss
    test_loss /= len(test_loader.dataset)
    # correct probability
    test_accuracy = 100.0 * correct / len(test_loader.dataset)

    return test_loss, test_accuracy

In [46]:
# train_baseline func
def train_baseline(model, train_loader, val_loader, optimizer, num_epochs=30) :
    print('call')
    best_acc = 0
    best_model_wts = copy.deepcopy(model.state_dict())
    
    for epoch in range(1, num_epochs + 1):
        since = time.time()
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)

        if val_acc > best_acc :
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since

        print(f'Train number [{epoch}]')
        print(f'[Train] - Loss  {train_loss} | Accuracy: {train_acc}')
        print(f'[Validation] - Loss  {val_loss} | Validation: {val_acc}')
        print(f'Train Time: {time_elapsed}')
        print('--------------')

    # after training, best weight model
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# train
# base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)
base = train_baseline(model_base, train_loader, val_loader, optimizer, 2)
torch.save(base, 'baseline.pt')

call
Train - [1/256]
Train - [2/256]
Train - [3/256]
Train - [4/256]
Train - [5/256]
Train - [6/256]
Train - [7/256]
Train - [8/256]
Train - [9/256]
Train - [10/256]
Train - [11/256]
Train - [12/256]
Train - [13/256]
Train - [14/256]
Train - [15/256]
Train - [16/256]
Train - [17/256]
Train - [18/256]
Train - [19/256]
Train - [20/256]
Train - [21/256]
Train - [22/256]
Train - [23/256]
Train - [24/256]
Train - [25/256]
Train - [26/256]
Train - [27/256]
Train - [28/256]
Train - [29/256]
Train - [30/256]
Train - [31/256]
Train - [32/256]
Train - [33/256]
Train - [34/256]
Train - [35/256]
Train - [36/256]
Train - [37/256]
Train - [38/256]
Train - [39/256]
Train - [40/256]
Train - [41/256]
Train - [42/256]
Train - [43/256]
Train - [44/256]
Train - [45/256]
Train - [46/256]
Train - [47/256]
Train - [48/256]
Train - [49/256]
Train - [50/256]
Train - [51/256]
Train - [52/256]
Train - [53/256]
Train - [54/256]
Train - [55/256]
Train - [56/256]
Train - [57/256]
Train - [58/256]
Train - [59/256]
T

In [ ]:
# predict
test_dataset = ImageFolder(root='./splitted/test/', transform=transform_base)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
data_iter = iter(test_loader)
batch_x, batch_y = next(data_iter)

print("Feature (x):", batch_x)
print("Label (y):", batch_y)
print("Feature shape:", batch_x.shape)
print("Label shape:", batch_y.shape)

In [ ]:
# predict
test_dataset = ImageFolder(root='./splitted/test/', transform=transform_base)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

model = torch.load('baseline.pt')

with torch.no_grad() :
    result = []

    for data, target in test_loader :
        data = data.to(DEVICE)
        target = target.to(DEVICE)
        output = model(data)
        
        pred = output.max(1, keepdim=True)[1]
        pred = pred.cpu().numpy()
        result = result + pred.reshape(-1).tolist()

print(result)

In [ ]:
results_data = []
classes_dict = test_dataset.class_to_idx
classes_names = list(classes_dict.keys())

for r1 in result :
    n1 = classes_names[r1]
    results_data.append(n1)

print(results_data)